#Installs

In [ ]:
!pip install -qU \
langchain \
langchain_core \
torch \
InstructorEmbedding \
chromadb \
huggingface-hub \
pypdf==3.7.1 \
tokenizers==0.14.0 \
sentence-transformers==2.2.2 \
tiktoken \
streamlit \
streamlit_chat \
streamlit_extras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 kB 33.2 MB/s eta 0:00:00


In [ ]:
# get GPU to run LLLM
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 92.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.38-cp310-cp310-manylinux_2_35_x86_64.whl size=9660813 sha256=7d175d205d7d6c963cfdec39994fb54ad28e2a32a67f894f667d0c608db558dc
  Stored in directory: /root/.cache/pip/wheels/eb/58/77/20d3d9a235b4930050fbcde1ad4f0a4d054644269e801b08aa
Successfully built llama-cpp-python


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.786s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Path to your PCB folder in Google Drive
DRIVE_FOLDER = '/content/drive/My Drive/PCB/EN_docs'

Mounted at /content/drive


# auxiliary.py

In [ ]:
%%writefile auxiliary.py

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.vectorstores import VectorStoreRetriever
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.documents.base import Document
from langchain_community.document_transformers import LongContextReorder
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.base import _get_chat_history
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
    Callbacks,
)

import torch
import inspect
from sentence_transformers import CrossEncoder
from typing import List, Dict, Any, Optional


device = 'cuda' if torch.cuda.is_available() else 'cpu'

########################
# MultiQuery Retriever #
########################

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

MULTIQUERY_TEMPLATE = """<s>[INST] You are an AI language model assistant.
Your task is to generate 3 different search queries that aim to answer the user question from multiple perspectives.
The user questions are focused on Banking, Finance, and related disciplines.
Each query MUST tackle the question from a different viewpoint, we want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines. [/INST]

Original question: {question}
"""

MULTIQUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template=MULTIQUERY_TEMPLATE,
    )


####################################
# Expansion with Answer Generation #
####################################

QUERY_EXPANSION_TEMPLATE = """<s>[INST]
Offer a concise and factual answer to the question. The response should be brief and without conversational tone, empathy, greetings, or personal comments.
[/INST]

QUESTION: {new_question}

Possible Answer:

"""

QUERY_EXPANSION_PROMPT = PromptTemplate(
    input_variables=["new_question"],
    template=QUERY_EXPANSION_TEMPLATE,
)

# Retriever that processes the initial query through an LLM Chain
class CustomRetriever(VectorStoreRetriever):
    def __init__(self, vectorstore: VectorStoreRetriever, chain: LLMChain, **kwargs):
        super().__init__(vectorstore=vectorstore, **kwargs)
        self.vectorstore = vectorstore
        self.chain = chain

    def get_relevant_documents(self, query: str) -> List[Document]:
        output = self.chain.invoke(query)
        new_query = output['new_question'] + "\n" + output['query']
        results = self.vectorstore.get_relevant_documents(query=new_query)
        return results

#######################
# Follow-up Question #
#######################

QUESTION_GENERATOR_TEMPLATE = """<s>[INST]
Use the chat history to clarify and simplify the follow-up question, ensuring it remains true to the original intent without adding extra details.
[/INST]

Chat History: {chat_history}

Follow-up Question: {question}

Refined Standalone Question:

"""

QUESTION_GENERATOR_PROMPT = PromptTemplate(
    template=QUESTION_GENERATOR_TEMPLATE,
    input_variables=["chat_history", "question"],
    )

#############################
# ChatBot Answer Generation #
#############################

ANSWER_TEMPLATE_BDP = """<s>[INST]
Role:
-Act as an AI chat assistant that has been trained exclusively to answer questions about the Bank Customer Website.
-Your goal is to respond to the users' questions using ONLY the information present on the provided [CONTEXT] below.

Rules:
-Make a step by step reasoning about the question.
-The response should be concise and clear.
-Don't mention that you have access to [CONTEXT] or a list of facts.
-You should always refuse to answer questions that are not related to [CONTEXT].
-You SHOULD NOT give any advice or recommendation to the user.
-If the question is out of the [CONTEXT] domain you should briefly and politely respond to the user that you don't have the information to provide the answer.
[/INST]

[CONTEXT]:
{context}

[QUESTION]:
{question}

Helpful Answer:

"""

ANSWER_PROMPT_BDP = PromptTemplate(
        template=ANSWER_TEMPLATE_BDP, input_variables=["context", "question"]
    )

ANSWER_TEMPLATE_DEFAULT = """<s>[INST] Use the following pieces of context to answer the question at the end.
If the context doesn't provide enough information to answer the question, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible. [/INST]

Context:
{context}

Question:
{question}

Helpful Answer:

"""

ANSWER_PROMPT_DEFAULT = PromptTemplate(
        template=ANSWER_TEMPLATE_DEFAULT, input_variables=["context", "question"]
    )

###################################################
# Custom Chain with follow-up question generation #
###################################################

def reranking(reranker_type, retrieved_docs, question):

    if reranker_type == "LongContextReorder":
        reranker = LongContextReorder()
        reranked_docs = reranker.transform_documents(retrieved_docs)

    elif reranker_type == "ms-marco-MiniLM-L-6-v2":
        cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
        # Get scores for each document
        scores = cross_encoder.predict([[question, doc.page_content] for doc in retrieved_docs])
        # Sort the documents by their scores
        reranked_docs = [doc for doc, score in sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)]

    elif reranker_type == "bge-reranker-base":
        cross_encoder = CrossEncoder('BAAI/bge-reranker-base', device=device)
        # Get scores for each document
        scores = cross_encoder.predict([[question, doc.page_content] for doc in retrieved_docs])
        # Sort the documents by their scores
        reranked_docs = [doc for doc, score in sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)]

    elif reranker_type == "None":
        reranked_docs = retrieved_docs

    return reranked_docs

class CustomConversationalRetrievalChain(ConversationalRetrievalChain):
    reranker_type: Optional[str] = Field(default=None, description="The type of reranker to use")

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
        ) -> Dict[str, Any]:
          _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
          question = inputs["question"]
          get_chat_history = self.get_chat_history or _get_chat_history
          chat_history_str = get_chat_history(inputs["chat_history"])

          callbacks = _run_manager.get_child()
          new_question = self.question_generator.run(
              question=question, chat_history=chat_history_str, callbacks=callbacks
          )
          accepts_run_manager = (
              "run_manager" in inspect.signature(self._get_docs).parameters
          )
          if accepts_run_manager:
              docs = self._get_docs(new_question, inputs, self.reranker_type, run_manager=_run_manager)
          else:
              docs = self._get_docs(new_question, inputs, self.reranker_type)  # type: ignore[call-arg]
          output: Dict[str, Any] = {}
          if self.response_if_no_docs_found is not None and len(docs) == 0:
              output[self.output_key] = self.response_if_no_docs_found
          else:
              new_inputs = inputs.copy()
              if self.rephrase_question:
                  new_inputs["question"] = new_question
              new_inputs["chat_history"] = chat_history_str
              answer = self.combine_docs_chain.run(
                  input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs
              )
              output[self.output_key] = answer

          if self.return_source_documents:
              output["source_documents"] = docs
          if self.return_generated_question:
              output["generated_question"] = new_question
          return output

    def _get_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        reranker_type: str,
        run_manager: CallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""
        retrieved_docs = self.retriever.get_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        docs = reranking(self.reranker_type, retrieved_docs, question)
        return self._reduce_tokens_below_limit(docs)

Writing auxiliary.py


# chatbot.py

In [ ]:
%%writefile chatbot.py

#################################################
# Streamlit App
#################################################

import io
import os
import json
import time
import random
import urllib
import itertools
import torch
import tiktoken
import transformers
import pandas as pd
import pypdf
import altair as alt

import streamlit as st
from streamlit_chat import message
from streamlit_extras.colored_header import colored_header
from streamlit_extras.add_vertical_space import add_vertical_space

from sentence_transformers import CrossEncoder
from urllib.error import URLError

from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain, RetrievalQA
from langchain.evaluation.qa import QAEvalChain, QAGenerateChain
from langchain.embeddings import HuggingFaceInstructEmbeddings, LlamaCppEmbeddings, OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.retrievers import BM25Retriever, EnsembleRetriever, MultiQueryRetriever
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain_community.document_transformers import LongContextReorder

from auxiliary import device, CustomRetriever, QUERY_EXPANSION_PROMPT, LineListOutputParser, LineList, MULTIQUERY_PROMPT, QUESTION_GENERATOR_PROMPT, ANSWER_PROMPT_BDP, ANSWER_PROMPT_DEFAULT, CustomConversationalRetrievalChain, reranking

# Set Streamlit page configuration
st.set_page_config(page_title='🧠BCW - ChatBot🤖', layout='wide')
st.header("ChatBot🤖: BdP - Bank Customer Website")
st.subheader("Powered by 🦜 LangChain + 👑 Streamlit")

def get_query_params():
    query_params = st.query_params

    params = {
        'oai_api_key': query_params.get('oai_api_key', 'None'),
        'chunk_size': int(query_params.get('chunk_size', '256')),
        'chunk_overlap': int(query_params.get('chunk_overlap', '20')),
        'retriever_type': query_params.get('retriever_type', 'Similarity'),
        'num_neighbors': int(query_params.get('num_neighbors', '3')),
        'hybrid_weight': float(query_params.get('hybrid_weight', '0.0')),
        'reranker_type': query_params.get('reranker_type', 'LongContextReorder'),
        'embeddings': query_params.get('embeddings', 'bge-base-en-v1.5'),
        'model': query_params.get('model', 'Mistral-7B-Instruct-v0.1 Small')
    }
    return params

# Retrieve and display parameters
params = get_query_params()
st.markdown("### 🛠️ Configured Parameters")
# Create a grid layout for better visualization
cols = st.columns(2)
for i, (key, value) in enumerate(params.items()):
    with cols[i % 2]:
        st.markdown(f"**{key.capitalize()}**: `{value}`")


# Assign parameters to variables
oai_api_key = params['oai_api_key']
chunk_size = params['chunk_size']
chunk_overlap = params['chunk_overlap']
retriever_type = params['retriever_type']
num_neighbors = params['num_neighbors']
hybrid_weight = params['hybrid_weight']
embeddings = params['embeddings']
reranker_type = params['reranker_type']
model = params['model']


def read_json_file(file_path):
    """
    Reads a JSON file and returns its contents.
    @param file_path: The path of the file to read.
    @return: The contents of the JSON file or None if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except IOError as e:
        st.error(f"Error reading file {file_path}: {e}")
        return None

def enrich_document(entry, doc_id):
    """
    Creates and enriches a Document object from a JSON entry.
    @param entry: The JSON entry containing document data.
    @param doc_id: The ID to assign to the document.
    @return: The enriched Document object or None if entry is invalid.
    """
    answer = entry.get('answer', '')
    if answer == '':
        return None
    source = entry.get('link', '')
    topic = entry.get('question') or entry.get('question_topic', '')
    if source == "https://clientebancario.bportugal.pt/en/perguntas-frequentes":
        topic_for_content = topic.split(" -> ")[-1]
        page_content = f"{topic_for_content}\n\n{answer}"
    else:
        page_content = f"{topic}\n\n{answer}"
    return Document(page_content=page_content, metadata={"source": source, "topic": topic, "doc_id": doc_id})

def update_level_2_documents(documents):
    """
    Updates level 2 documents with corresponding level 3 topics.
    @param documents: A list of Document objects.
    @return: None. The function modifies the documents list in place.
    """
    level_2_docs = {}
    level_3_topics = []

    # Categorize documents based on their level
    for doc in documents:
        topic = doc.metadata.get('topic', '')
        level = topic.count('->') + 1
        if level == 2:
            level_2_docs[topic] = doc
        elif level == 3:
            level_3_topics.append(topic)

    # Update level 2 documents if they have corresponding level 3 topics
    for level_2_topic, level_2_doc in level_2_docs.items():
        if level_2_doc.page_content.endswith(':'):
            # Extract the base of the level 2 topic for matching with level 3 topics
            base_topic = level_2_topic + ' -> '
            matching_topics = [t.split(' -> ')[-1] for t in level_3_topics if t.startswith(base_topic)]
            if matching_topics:
                level_2_doc.page_content += '\n' + '\n'.join(matching_topics)

def get_documents(folder_path="/content/drive/My Drive/PCB/EN_docs"):
    """
    Loads and processes documents from a specified folder.
    @param folder_path: The path of the folder containing the documents.
    @return: A list of processed Document objects.
    """
    documents = []
    doc_id = 0

    for file_name in sorted(os.listdir(folder_path)):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            json_data = read_json_file(file_path)
            if json_data:
                for entry in json_data.get('content', []):
                    doc = enrich_document(entry, doc_id)
                    if doc:
                        documents.append(doc)
                        doc_id += 1

    # Update level 2 documents with corresponding level 3 topics
    update_level_2_documents(documents)

    return documents

def token_len(text):
    tokenizer = tiktoken.get_encoding('cl100k_base')
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

def get_chunks(documents, chunk_size, overlap):
    """
    Split documents into chunks
    @param text: documents to split
    @param chunk_size: charecters per split
    @param overlap: charecter overlap between chunks
    @return: list of chunks
    """

    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""],
                                                chunk_size=chunk_size,
                                                chunk_overlap=overlap,
                                                length_function=token_len)
    chunks = []
    for doc in documents:
        page_content = doc.page_content
        metadata = doc.metadata
        chunked_doc = text_splitter.create_documents(
            texts=[page_content],
            metadatas=[metadata]
        )
        for chunked_text in chunked_doc:
            chunked_metadata = chunked_text.metadata
            # remove chunks that only have the topic or the FAQ
            if chunked_text.metadata['source'] == "https://clientebancario.bportugal.pt/en/perguntas-frequentes":
                topic = chunked_text.metadata['topic'].split(" -> ")[-1]
            else:
                topic = chunked_text.metadata['topic']
            if chunked_text.page_content.strip() == topic.strip():
                continue
            else:
                new_chunk = Document(page_content=chunked_text.page_content, metadata=chunked_metadata)
                chunks.append(new_chunk)

    return chunks


def load_mistral(mistral_model):
    """
    Load mistral model from path
    @mistral_model: mistral model to load
    """

    if mistral_model == "Mistral-7B-Instruct-v0.1 Small":
        model_path = 'mistral-7b-instruct-v0.1.Q2_K.gguf'

    elif mistral_model == "Mistral-7B-Instruct-v0.1 Med":
        model_path = 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'

    llm = LlamaCpp(
          model_path=f"/content/drive/My Drive/PCB/mistral/{model_path}",
          temperature=0.0,
          f16_kv=True,
          top_p=1,
          n_ctx=4096,
          n_batch=1024,
          n_gpu_layers=100)

    return llm

def get_llm(model):
    """
    Get LLM
    @param model: LLM to use
    @return: LLM
    """

    if model == 'GPT-4-turbo':
        llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

    elif model == "Mistral-7B-Instruct-v0.1 Small" or model == "Mistral-7B-Instruct-v0.1 Med":
        llm = load_mistral(model)

    return llm


def get_retriever(chunks, retriever_type, embedding_type, num_neighbors, llm, hybrid_weight):
    """
    Get document retriever
    @param chunks: list of Documents (chunks)
    @param retriever_type: retriever type
    @param embedding_type: embedding type
    @param num_neighbors: number of neighbors for retrieval
    @return: retriever
    """

    # Set embeddings
    if embedding_type == "OpenAI":
        embd = OpenAIEmbeddings(model="text-embedding-3-small")

    elif embedding_type in ["bge-base-en-v1.5", "bge-small-en-v1.5", "bge-large-en-v1.5"]:
        model_name = f"BAAI/{embedding_type}"
        embd = HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs={"device": device})

    # Select retriever
    if retriever_type == "Similarity":
        vectorstore = Chroma.from_documents(chunks, embd)
        retriever = vectorstore.as_retriever(search_type='similarity_score_threshold', search_kwargs={"k": num_neighbors, "score_threshold": 0.4})

    if retriever_type == "BM25":
        retriever = BM25Retriever.from_documents(chunks)

    elif retriever_type == "Hybrid-search":
        vectorstore = Chroma.from_documents(chunks, embd)
        chroma_retriever = vectorstore.as_retriever(search_type='similarity_score_threshold', search_kwargs={"k": num_neighbors, "score_threshold": 0.4})
        bm25_retriever = BM25Retriever.from_documents(chunks)
        retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever],weights=[hybrid_weight,1-hybrid_weight])

    elif retriever_type == "Similarity + Expansion w/ Generated Answers":
        vectorstore = Chroma.from_documents(chunks, embd)
        chain = LLMChain(llm=llm, prompt=QUERY_EXPANSION_PROMPT, output_key="query")
        retriever = CustomRetriever(vectorstore=vectorstore.as_retriever(search_type='similarity_score_threshold', search_kwargs={"k": num_neighbors, "score_threshold": 0.4}), chain=chain)
        mq_retrieval_chain = LLMChain(llm=llm, prompt=MULTIQUERY_PROMPT, output_parser=LineListOutputParser())

    elif retriever_type == "MultiQuery Retriever":
        vectorstore = Chroma.from_documents(chunks, embd)
        chain = LLMChain(llm=llm, prompt=MULTIQUERY_PROMPT, output_parser=LineListOutputParser())
        retriever = MultiQueryRetriever(
                    retriever=vectorstore.as_retriever(search_type='similarity_score_threshold', search_kwargs={"k": num_neighbors, "score_threshold": 0.4}),
                    llm_chain=mq_retrieval_chain,
                    parser_key="lines",
                    include_original=True
                    )

    return retriever


# Run App

if 'submitted' not in st.session_state:
    st.session_state.submitted = False

# Document Upload Section - Shown only if not submitted
if not st.session_state.submitted:
    st.subheader('Document Upload')
    uploaded_docs = st.file_uploader("Upload documents (.pdf or .txt):", type=['pdf', 'txt'], accept_multiple_files=True)
    use_bp_example = st.checkbox("Use Banco de Portugal - Bank Customer Website")
    submit_selections = st.button('Submit Selections')
    if submit_selections:
        st.session_state.submitted = True
        st.session_state.uploaded_docs = uploaded_docs
        st.session_state.use_bp_example = use_bp_example

# Processing after document submission
if st.session_state.submitted:

    with st.spinner("Loading ChatBot..."):

        if st.session_state.use_bp_example:
            documents = get_documents()
            chunks = get_chunks(documents, chunk_size, chunk_overlap)
            prompt = ANSWER_PROMPT_BDP

        if st.session_state.uploaded_docs:
            combined_text, fnames = [], []
            for file in sorted(st.session_state.uploaded_docs):
                contents = file.read()
                # PDF file
                if file.type == 'application/pdf':
                    pdf_reader = pypdf.PdfReader(io.BytesIO(contents))
                    text = ""
                    for page in pdf_reader.pages:
                        text += page.extract_text()
                    combined_text.append(text)
                    fnames.append(file.name)
                # Text file
                elif file.type == 'text/plain':
                    combined_text.append(contents.decode())
                    fnames.append(file.name)
                else:
                    st.warning("Unsupported file type for file: {}".format(file.name))
            text = " ".join(combined_text)
            text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""], chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            texts = text_splitter.split_text(text)
            chunk_ids = [{"chunk_id": i} for i in range(len(texts))]
            chunks = text_splitter.create_documents(texts, chunk_ids)
            prompt = ANSWER_PROMPT_DEFAULT

        # Get LLM
        if 'llm' not in st.session_state:
            st.session_state.llm = get_llm(model)

        # Get retriver
        if 'retriever' not in st.session_state:
            st.session_state.retriever = get_retriever(chunks, retriever_type, embeddings, num_neighbors, st.session_state.llm, hybrid_weight)

if 'llm' in st.session_state:
    memory = ConversationBufferWindowMemory(k=4, memory_key="chat_history", input_key="question", output_key="answer", return_messages=True)

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []
    else:
        for chat in st.session_state.chat_history:
            memory.save_context({'question': chat['human']}, {'answer': chat['AI']})

    chain = CustomConversationalRetrievalChain.from_llm(
          llm=st.session_state.llm,
          memory=memory,
          combine_docs_chain_kwargs={"prompt": prompt},
          condense_question_prompt=QUESTION_GENERATOR_PROMPT,
          retriever=st.session_state.retriever,
          reranker_type=reranker_type,
          condense_question_llm=st.session_state.llm,
          chain_type="stuff",
          output_key="answer",
          response_if_no_docs_found="I am sorry, I don't have available information to answer that question.",
          verbose=True,
          return_generated_question=True,
          return_source_documents=True,
          )

    if "user_input" not in st.session_state:
        st.session_state.user_input = ""

    def submit():
        st.session_state.user_input = st.session_state.widget
        st.session_state.widget = ""

    st.text_input("You: ",
                  key="widget",
                  placeholder="Hello, I am your AI assistant! How can I help you?",
                  label_visibility='hidden',
                  on_change=submit)

    user_input = st.session_state.user_input

    if user_input:
        with st.spinner("Generating answer..."):
            output = chain.invoke(input=user_input)
            answer = output["answer"]
            sources = output.get("source_documents", [])

        # Only show sources for the current (last) output
        if sources:
            with st.expander("Sources used in Answer", expanded=False):
                for i, source in enumerate(sources, start=1):
                    st.markdown(f"**Source {i}:**\n{source.page_content}")

        st.session_state.chat_history.append({'human': user_input, 'AI': answer})

    messages = st.session_state.get("chat_history", [])
    # Display messages in reverse order
    for msg in reversed(messages):
        if 'human' in msg and msg['human']:
            message(msg['human'], is_user=True, key=str(msg) + "_user")
        if 'AI' in msg and msg['AI']:
            message(msg['AI'], is_user=False, key=str(msg) + "_ai")

Overwriting chatbot.py


# Run App

In [ ]:
!streamlit run chatbot.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.124.215.20


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.806s
your url is: https://many-dancers-sin.loca.lt
^C
